# **Laboratorio 9**
**Daniela Navas**

## **Task 1** - Teoría

**Responda las siguientes preguntas de forma clara y concisa, pueden subir un PDF o bien dentro del mismo Jupyter Notebook.**<br> 
**1. Diga cual es la diferencia entre Modelos de Markov y Hidden Markov Models**<br>
En los **Modelos de Markov** los estados son directamente observables. La probabilidad de transición entre estados depende únicamente del estado actual, no de los estados anteriores. Se utilizan comúnmente en aplicaciones donde los estados son visibles, como la predicción del clima o la generación de texto. En cambio, en los **Hidden Markov Models (HMM)** los estados no son directamente observables (son "ocultos"). En cambio, se observan emisiones que dependen de estos estados ocultos. Los HMM son útiles en situaciones donde los estados subyacentes no pueden ser observados directamente, como en el reconocimiento de voz o la bioinformática.

**2. Investigue qué son los factorial HMM (Hidden Markov Models)**<br>
Los factorial HMM son una extensión de los HMM estándar. En lugar de tener una sola cadena de estados ocultos, los factorial HMM tienen múltiples cadenas de estados ocultos que interactúan entre sí. Esto permite modelar sistemas más complejos donde múltiples factores pueden influir en las observaciones. Se utilizan en aplicaciones como el análisis de datos multivariados y la modelación de sistemas biológicos.

**3. Especifique en sus propias palabras el algoritmo Forward Backward para HMM**<br>
El algoritmo Forward Backward es un método de inferencia para HMM que calcula las probabilidades posteriores de los estados ocultos dados una secuencia de observaciones. Se realiza en dos pasos:
1. **Forward:** Calcula las probabilidades de estar en cada estado en cada punto de tiempo, dado las observaciones hasta ese punto.
2. **Backward:** Calcula las probabilidades de observar las secuencias restantes desde cada punto de tiempo hacia adelante. Al combinar estas probabilidades, se obtiene la distribución de los estados en cualquier punto de tiempo dado toda la secuencia de observaciones.

**4. En el algoritmo de Forward Backward, por qué es necesario el paso de Backward (puede escribir ejemplos o casos para responder esta pregunta)**<br>
El paso de Backward es crucial porque permite incorporar información futura en el cálculo de las probabilidades de los estados actuales. Sin este paso, solo se tendría en cuenta la información pasada y presente, lo que podría llevar a estimaciones menos precisas. Por ejemplo, si estás modelando el reconocimiento de voz, la información sobre las palabras futuras puede ayudar a determinar mejor los estados actuales de los fonemas.

---


---

**GITHUB:**
https://github.com/danielanavas2002/InteligenciaArtificial/tree/main/Laboratorio/Laboratorio09